In [2]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from pymongo import MongoClient

import pprint

import pandas as pd

# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup

import json



chrome_driver_path = "/Users/annebarry/galvanize/running_rivalries/chromedriver"

driver = webdriver.Chrome(executable_path = chrome_driver_path)


driver.get('https://results.nyrr.org/event/20VGRD1/finishers#af=40&at=44')

In [3]:
## get results

button = driver.find_elements_by_class_name('button-load-more')

while True:
    try:
        button[1].click()
        time.sleep(10)
    except:
        break

m_40_44 = driver.page_source

In [18]:
##sets up mongo DB
client = MongoClient('localhost', 27017)
races = client['new_db']
race_results = races['race_results']
individual_results = races['individual']

In [19]:
races.race_results.insert({'name':'Marathon Men 40-44', 'source': m_40_44})

/Users/annebarry/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


ObjectId('5f160476384b124043ec55ab')

In [20]:
races.race_results.find()

In [21]:
def parse_results(data, distance, ag):
    soup = BeautifulSoup(data, 'html.parser')
    for i, result in enumerate(soup.findAll("div",'rms-grid-item')):
        t = result.findAll("span", "long-text")[0].text[4:]
        a = int(result.findAll("span", "ng-binding ng-scope")[0].text[1:])
        n = result.findAll('div', 'name rms-grid-line ng-binding')[0].text.strip()
        individual_results.insert({'name': n, 'time':t, 'age': a, 'ag': ag, 'dist': distance})
parse_results(m_40_44, 'Mar', '40-44')

/Users/annebarry/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  import sys


In [22]:
races.individual_results.find().limit(4)

In [23]:
data = pd.DataFrame(list(individual_results.find()))

In [24]:
print(data)

                           _id                name     time  age     ag dist
0     5f1604e7384b124043ec55ac          Clint Bell  0:04:49   40  40-44  Mar
1     5f1604e7384b124043ec55ad  Katherine Phillips  0:05:03   40  40-44  Mar
2     5f1604e7384b124043ec55ae       Keith Stewart  0:05:05   40  40-44  Mar
3     5f1604e7384b124043ec55af     Matthew Jacklin  0:05:05   44  40-44  Mar
4     5f1604e7384b124043ec55b0        Brian Pasero  0:05:05   41  40-44  Mar
...                        ...                 ...      ...  ...    ...  ...
2358  5f1604ec384b124043ec5ee2           Elise Hay  0:26:46   41  40-44  Mar
2359  5f1604ec384b124043ec5ee3          Susan Abla  0:27:01   42  40-44  Mar
2360  5f1604ec384b124043ec5ee4       Silvia Garcia  0:27:19   41  40-44  Mar
2361  5f1604ec384b124043ec5ee5       Emma Millward  0:31:57   40  40-44  Mar
2362  5f1604ec384b124043ec5ee6      Cordelia Stark  2:28:04   44  40-44  Mar

[2363 rows x 6 columns]
